### Importing libraties

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

### Define parameters

In [2]:
filepath = 'bumpgrid.dat.txt'

gamma = 1.4

rho_inf = 1.225 
Mach    = 1.4 
P_inf   = 1.0*1e+5
a_inf   = np.sqrt(gamma*P_inf/rho_inf)  # a_inf = sqrt(gamma p / rho) = 1
u_inf   = Mach*a_inf                   
v_inf   = 0.0       

CFL     = 0.5
tol     = 1e-4
niter   = 5000


### Reading Dataset and creating geometry

In [3]:
data = np.loadtxt(filepath)
data = np.array(data)

[imx, jmx] = data[0, :]
imx = int(imx)
jmx = int(jmx)

X = np.zeros([imx, jmx])
Y = np.zeros([imx, jmx])

for j in range(jmx):
    for i in range(imx):
        X[i, j] = data[j * imx + i + 1, 0]
        Y[i, j] = data[j * imx + i + 1, 1]

In [4]:
xnx = np.zeros([imx + 1, jmx + 1])
xny = np.zeros([imx + 1, jmx + 1])

# i-direction faces
for j in range(1, jmx):
    for i in range(imx):
        xnx[i, j] = Y[i, j] - Y[i, j - 1]
        xny[i, j] = -(X[i, j] - X[i, j - 1])

ynx = np.zeros([imx + 1, jmx + 1])
yny = np.zeros([imx + 1, jmx + 1])

for j in range(jmx):
    for i in range(1, imx):
        ynx[i, j] = Y[i - 1, j] - Y[i, j]
        yny[i, j] = -(X[i - 1, j] - X[i, j])

vol = np.zeros([imx + 1, jmx + 1])

for j in range(1, jmx):
    for i in range(1, imx):
        x_1, y_1 = X[i - 1, j - 1], Y[i - 1, j - 1]
        x_2, y_2 = X[i, j - 1], Y[i, j - 1]
        x_3, y_3 = X[i, j], Y[i, j]
        x_4, y_4 = X[i - 1, j], Y[i - 1, j]
        vol[i, j] = 0.5 * (
            abs((x_2 - x_1) * (y_4 - y_2) - (y_2 - y_1) * (x_4 - x_1)) +
            abs((x_4 - x_3) * (y_2 - y_3) - (y_4 - y_3) * (x_2 - x_3))
        )


### Create functions to convert functions

In [ ]:
def conservative_to_primitive(rho, rhou, rhov, rhoE, gamma=1.4):
    u = rhou/rho
    v = rhov/rho
    velocity = np.sqrt(u**2 + v**2)
    P = (gamma - 1) * (rhoE - 0.5 * rho * velocity**2)
    return u, v, P

def primitive_to_conservative(rho, u, v, P, gamma=1.4):
    # convert primitive variables to conservative variables
    velocity = np.sqrt(u**2 + v**2)
    E = P/((gamma + 1)*rho) + 0.5 * velocity**2
    return rho, rho*u, rho*v, rho*E


In [ ]:
def Roe_Dissipation(q_L, q_R,n_x,n_y,gamma=1.4):

    Area = math.sqrt(n_x**2 + n_y**2)
    nx_hat = n_x/Area
    ny_hat = n_y/Area

    # Left State
    rho_L = q_L[0]
    ul = q_L[1]/rho_L
    vl = q_L[2]/rho_L
    Pl = (gamma - 1) * (q_L[3] - 0.5 * rho_L * (ul**2 + vl**2))
    h_L = (q_L[3] + Pl)/rho_L

    # Right State
    rho_R = q_R[0]
    ur = q_R[1]/rho_R
    vr = q_R[2]/rho_R
    Pr = (gamma - 1) * (q_R[3] - 0.5 * rho_R * (ur**2 + vr**2))
    h_R = (q_R[3] + Pr)/rho_R

    # Normal Velocities
    vnl = ul*nx_hat + vl*ny_hat
    vnr = ur*nx_hat + vr*ny_hat

    # Roe Averages
    r = math.sqrt(rho_R/rho_L)
    rho_R = math.sqrt(rho_L*rho_R)
    u_R = (ul + r*ur)/(1 + r)
    v_R = (vl + r*vr)/(1 + r)
    h0_R = (h_L + r*h_R)/(1 + r)
    a_R = math.sqrt((gamma - 1)*(h0_R - 0.5*(u_R**2 + v_R**2)))
    vn_R = u_R*nx_hat + v_R*ny_hat

    d_rho = rho_R - rho_L
    d_P = Pr - Pl
    d_vn = vnr - vnl
    d_u = ur - ul
    d_v = vr - vl

    # Construct alpha coefficients
    alpha_1 = Area*abs(vn_R)*(d_rho - d_P/(a_R**2))
    alpha_2 = Area/(2*a_R**2)*abs(vn_R+a_R)*(d_P + rho_R*a_R*d_vn)
    alpha_3 = Area/(2*a_R**2)*abs(vn_R-a_R)*(d_P - rho_R*a_R*d_vn)
    alpha_4 = alpha_1 + alpha_2 + alpha_3
    alpha_5 = a_R*(alpha_2-alpha_3)
    alpha_6 = Area*abs(vn_R)*(rho_R*d_u - nx_hat*rho_R*d_vn)
    alpha_7 = Area*abs(vn_R)*(rho_R*d_v - ny_hat*rho_R*d_vn)

    # Dissipation Fluxes
    d_1 = alpha_4
    d_2 = alpha_4*u_R + alpha_5*nx_hat + alpha_6
    d_3 = alpha_4*v_R + alpha_5*ny_hat + alpha_7
    d_4 = alpha_4*h0_R + alpha_5*vn_R + alpha_6*u_R + alpha_7*v_R - a_R*alpha_1/(gamma - 1)

    return np.array([d_1, d_2, d_3, d_4])

### Function for computing fluxes on face normals

In [ ]:
def save_solution(iter_no, rho, rhou, rhov, rhoE, gamma, X, Y):
    